In [1]:
from utils.s3 import S3Manager
import pandas as pd

In [2]:
!pip install xgboost
import pandas as pd
import pickle as pickle
from datetime import datetime
import numpy as np
import pkg_resources
from scipy.stats import uniform, randint
import matplotlib.pyplot as plt
import math
from math import log
import sklearn as sklearn
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import r2_score
from xgboost import plot_importance

pd.options.display.html.table_schema=True #Para ver con barra los dataframes


You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
team = 'team09'
s3_manager = S3Manager()

In [4]:
fs_campana = 's3://belc-datathon-{}/campana_consultora.csv'.format(team)
fs_exposicion = 's3://belc-datathon-{}/exposicion.csv'.format(team)
fs_consultora = 's3://belc-datathon-{}/maestro_consultora.csv'.format(team)
fs_venta = 's3://belc-datathon-{}/venta.csv'.format(team)

In [5]:
dfcampana = pd.read_csv(s3_manager.s3_getfile(fs_campana)['Body'],dtype ={'idconsultora':str})
dfexposicion = pd.read_csv(s3_manager.s3_getfile(fs_exposicion)['Body'])
dfconsultora = pd.read_csv(s3_manager.s3_getfile(fs_consultora)['Body'],dtype ={'idconsultora':str})
dfventa = pd.read_csv(s3_manager.s3_getfile(fs_venta)['Body'],dtype ={'idconsultora':str})

In [430]:
dfexposicion.groupby(["campana"]).count()

,marcacategoria,idmarcacategoria,medioventa,nropaginas,nrodeofertas,flagmodelo,flagproducto,flagbulk,flagcaratula,flagcontracaratula,flagofertadigitaltematica,flagofertapersonalizada,flagliquidacionweb
campana,,,,,,,,,,,,,
201801,116,116,116,116,116,116,116,116,116,116,116,116,116
201802,121,121,121,121,121,121,121,121,121,121,121,121,121
201803,130,130,130,130,130,130,130,130,130,130,130,130,130
201804,132,132,132,132,132,132,132,132,132,132,132,132,132
201805,115,115,115,115,115,115,115,115,115,115,115,115,115
201806,121,121,121,121,121,121,121,121,121,121,121,121,121
201807,133,133,133,133,133,133,133,133,133,133,133,133,133
201808,132,132,132,132,132,132,132,132,132,132,132,132,132
201809,134,134,134,134,134,134,134,134,134,134,134,134,134


In [6]:
dfventa['idconsultora'] = dfventa['idconsultora'].str.zfill(7)
dfventa.head()

,campana,idconsultora,marcacategoria,unidades,venta,idmarcacategoria
0,201814,0124824,ESIKA/NO COSMETICOS,1,84.73001,50
1,201816,0005990,ESIKA/NO COSMETICOS,1,108.93000,50
2,201810,0189544,CYZONE/MAQUILLAJE/PARA LABIOS,2,36.24000,20
3,201811,0159811,CYZONE/NO COSMETICOS,2,254.20000,31
4,201807,0133523,CYZONE/NO COSMETICOS,1,78.66001,31


In [7]:
dfventa["idconcat"] = dfventa["idconsultora"].astype(str) + "-" + dfventa["idmarcacategoria"].astype(str)
dfventa["idconcat2"] = dfventa["idconsultora"].astype(str) + "-" + dfventa["idmarcacategoria" ].astype(str) + "-" + dfventa["campana" ].astype(str)

In [161]:
# venta["idconcat2"] = venta["campana"].astype(str) + "-" + venta["idmarcacategoria"].astype(str)
# venta["Target"] = 1

In [164]:
# exposicion["idconcat2"] = exposicion["campana"].astype(str) + "_" + exposicion["idmarcacategoria"].astype(str)

In [168]:
# dat = pd.merge(exposicion[["idconcat2"]], venta ,how='outer', left_on=['idconcat2'], right_on=['idconcat2'])

In [8]:
del dfexposicion["marcacategoria"]
dat = pd.merge(dfexposicion, dfventa ,how='left', on=['idmarcacategoria','campana'])

In [9]:
dat = dat.dropna()
dat.isnull().sum()

campana                      0
idmarcacategoria             0
medioventa                   0
nropaginas                   0
nrodeofertas                 0
flagmodelo                   0
flagproducto                 0
flagbulk                     0
flagcaratula                 0
flagcontracaratula           0
flagofertadigitaltematica    0
flagofertapersonalizada      0
flagliquidacionweb           0
idconsultora                 0
marcacategoria               0
unidades                     0
venta                        0
idconcat                     0
idconcat2                    0
dtype: int64

In [11]:
len(dfexposicion.idmarcacategoria.unique()) 
len(dat.idmarcacategoria.unique()) 

55

In [12]:
###### CREACION DEL TARGET ######
dfventa["flagcompro"] = 0
dfventa["flagcompro"][(dfventa.marcacategoria == 'ESIKA/FRAGANCIAS/CABALLEROS')|
                 (dfventa.marcacategoria == 'CYZONE/FRAGANCIAS/CABALLEROS')|
                 (dfventa.marcacategoria == 'LBEL/FRAGANCIAS/CABALLEROS')|
                 (dfventa.marcacategoria == 'LBEL/FRAGANCIAS/DAMAS')|
                 (dfventa.marcacategoria == 'ESIKA/FRAGANCIAS/DAMAS')|
                 (dfventa.marcacategoria == 'CYZONE/FRAGANCIAS/DAMAS')|
                 (dfventa.marcacategoria == 'ESIKA/TRATAMIENTO FACIAL')|
                 (dfventa.marcacategoria == 'LBEL/TRATAMIENTO FACIAL')|
                 (dfventa.marcacategoria == 'CYZONE/TRATAMIENTO FACIAL')] = 1

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [373]:
sum(dfventa.flagcompro) / len(dfventa)

0.23418470554680973

In [13]:
dat["idconcat2"] = dat["idconsultora"].astype(str) + "-" + dat["idmarcacategoria" ].astype(str) + "-" + dat["campana" ].astype(str)

In [14]:
dat = pd.merge(dat, dfventa[["idconcat2", "flagcompro"]]  ,how='left', left_on=['idconcat2'], right_on=['idconcat2'])

In [428]:
dat[dat.idconsultora == '0532884'].sort_values('idmarcacategoria')

,campana,idmarcacategoria,medioventa,nropaginas,nrodeofertas,flagmodelo,flagproducto,flagbulk,flagcaratula,flagcontracaratula,flagofertadigitaltematica,flagofertapersonalizada,flagliquidacionweb,idconsultora,unidades,venta,idconcat,idconcat2,flagcompro
3593586,201818,24,OTROS,1,2,0,0,0,0,0,0,1,0,0532884,1.0,56.77,0532884-24,0532884-24-201818,0
5274355,201818,24,CATALOGO,3,7,1,1,0,0,0,0,0,0,0532884,1.0,56.77,0532884-24,0532884-24-201818,0
350946,201818,41,CATALOGO,8,19,1,1,0,0,0,0,0,0,0532884,3.0,102.87,0532884-41,0532884-41-201818,0
863680,201818,41,REVISTA,3,7,0,1,0,0,0,0,0,0,0532884,3.0,102.87,0532884-41,0532884-41-201818,0
4047749,201818,41,OTROS,1,25,0,0,0,0,0,1,1,1,0532884,3.0,102.87,0532884-41,0532884-41-201818,0
755373,201818,43,CATALOGO,5,56,1,1,1,0,0,0,0,0,0532884,3.0,96.82,0532884-43,0532884-43-201818,0
915826,201818,43,OTROS,1,13,0,0,0,0,0,1,0,1,0532884,3.0,96.82,0532884-43,0532884-43-201818,0
5550306,201818,43,REVISTA,1,37,0,1,1,0,0,0,0,0,0532884,3.0,96.82,0532884-43,0532884-43-201818,0
384272,201818,48,OTROS,2,105,0,1,0,0,0,1,1,1,0532884,2.0,186.10,0532884-48,0532884-48-201818,1
1050595,201818,48,CATALOGO,19,32,1,1,0,0,0,0,0,0,0532884,2.0,186.10,0532884-48,0532884-48-201818,1


In [379]:
sum(dat.flagcompro) / len(dat)

0.24980726503306083

In [ ]:
####### VENTAS Y EXPOSICIÓN

In [15]:
############### AGRUPADO ##############
#DEFINIR IDS PARA TRAIN Y TEST(ENVIO KAGGLE)
fs_submit = 's3://belc-datathon-{}/predict_submit.csv'.format(team)
dfsubmit = pd.read_csv(s3_manager.s3_getfile(fs_submit)['Body'],dtype ={'idconsultora':str})

In [16]:
ids = dfsubmit["idconcat"]

In [17]:
del dat["marcacategoria"]

In [18]:
dat2 = pd.merge(ids, dat  ,how='left', left_on=['idconcat'], right_on=['idconcat'])

In [19]:
ktest = dat2[(dat2.campana >= 201803) & (dat2.campana <= 201818)]
test = dat2[(dat2.campana >= 201802) & (dat2.campana <= 201817)]
train = dat2[(dat2.campana >= 201801) & (dat2.campana <= 201816)]

In [20]:
y_test = dat2["flagcompro"][(dat2.campana == 201818)]
y_train = dat2["flagcompro"][(dat2.campana == 201817)]

In [21]:
vars_miss = []
monotonas = []
discretas = []
num_discr = []
continuas = []

for i in train.columns:  
  if not i in ['idconcat','idconcat2','campana','idconsultora','Target','idmarcacategoria']:
    if (len(train[pd.isnull(train[i]) == True])/len(train)) >= 0.9:
      vars_miss.append(i)
    elif len(train[i].unique()) == 1:
      monotonas.append(i)  
    elif (len(train[i].unique()) > 1) & (len(train[i].unique()) < 12 ): 
      discretas.append(i)
      num_discr.append(len(train[i].unique()))
    elif len(train[i].unique()) >= 12: 
      continuas.append(i)

In [22]:
vec_cat = []

for i in range(len(discretas)):
  a = discretas[i]
  t = type((train[a][pd.isnull(train[a]) == False].reset_index(drop=True))[0])
  
  if (t is str):
      b = train[a].unique()   
      for j in range(len(b)-1):
          train[a + '_' + str(b[j])] = 0
          train[a + '_' + str(b[j])][(train[a] == b[j]) == True] = 1
          
          test[a + '_' + str(b[j])] = 0
          test[a + '_' + str(b[j])][(test[a] == b[j]) == True] = 1

          ktest[a + '_' + str(b[j])] = 0
          ktest[a + '_' + str(b[j])][(ktest[a] == b[j]) == True] = 1
        
          vec_cat.append(a + '_' + str(b[j]))  
      train.drop([a], axis='columns', inplace=True)
      test.drop([a], axis='columns', inplace=True)
      ktest.drop([a], axis='columns', inplace=True)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace

In [23]:
discretas = ['flagmodelo','flagproducto','flagbulk','flagcaratula','flagofertadigitaltematica','flagofertapersonalizada',
 'flagliquidacionweb','flagcompro']

c=['count','nunique']
n=['mean','max','min','sum','std']
n1=["sum"]
n2=['mean','sum','std']
n3=['max']
nn=['mean','max','min','sum','std','quantile']

agg_c={      
}

boxes = { 
   i : n2        
   for i in discretas    
} 
agg_c.update(boxes)  

boxes = {
   i : nn        
   for i in continuas    
}  
agg_c.update(boxes)  

boxes = {
   i : n2        
   for i in vec_cat    
}  
agg_c.update(boxes)  


In [24]:
tt = train[(train.campana >= 201815) & (train.campana <= 201816)].copy()

{'flagmodelo': ['mean', 'sum', 'std'],
 'flagproducto': ['mean', 'sum', 'std'],
 'flagbulk': ['mean', 'sum', 'std'],
 'flagcaratula': ['mean', 'sum', 'std'],
 'flagofertadigitaltematica': ['mean', 'sum', 'std'],
 'flagofertapersonalizada': ['mean', 'sum', 'std'],
 'flagliquidacionweb': ['mean', 'sum', 'std'],
 'flagcompro': ['mean', 'sum', 'std'],
 'nropaginas': ['mean', 'max', 'min', 'sum', 'std', 'quantile'],
 'nrodeofertas': ['mean', 'max', 'min', 'sum', 'std', 'quantile'],
 'unidades': ['mean', 'max', 'min', 'sum', 'std', 'quantile'],
 'venta': ['mean', 'max', 'min', 'sum', 'std', 'quantile'],
 'medioventa_OTROS': ['mean', 'sum', 'std'],
 'medioventa_REVISTA': ['mean', 'sum', 'std'],
 'medioventa_CATALOGO': ['mean', 'sum', 'std']}

In [41]:
train = train.groupby(['idconcat']).agg(agg_c)
train.columns=['CV_' + '_'.join(col).strip() for col in train.columns.values]
train.reset_index(inplace=True)

test = test.groupby(['idconcat']).agg(agg_c)
test.columns=['CV_' + '_'.join(col).strip() for col in test.columns.values]
test.reset_index(inplace=True)

ktest = test.groupby(['idconcat']).agg(agg_c)
ktest.columns=['CV_' + '_'.join(col).strip() for col in test.columns.values]
ktest.reset_index(inplace=True)



KeyError: "Column 'flagmodelo' does not exist!"

In [39]:
tt = tt.groupby(['idconcat']).agg(agg_c)
tt.columns=['CV_' + '_'.join(col).strip() for col in tt.columns.values]
tt.reset_index(inplace=True)

In [40]:
tt.keys()

Index(['idconcat', 'CV_flagmodelo_mean', 'CV_flagmodelo_sum',
       'CV_flagmodelo_std', 'CV_flagproducto_mean', 'CV_flagproducto_sum',
       'CV_flagproducto_std', 'CV_flagbulk_mean', 'CV_flagbulk_sum',
       'CV_flagbulk_std', 'CV_flagcaratula_mean', 'CV_flagcaratula_sum',
       'CV_flagcaratula_std', 'CV_flagofertadigitaltematica_mean',
       'CV_flagofertadigitaltematica_sum', 'CV_flagofertadigitaltematica_std',
       'CV_flagofertapersonalizada_mean', 'CV_flagofertapersonalizada_sum',
       'CV_flagofertapersonalizada_std', 'CV_flagliquidacionweb_mean',
       'CV_flagliquidacionweb_sum', 'CV_flagliquidacionweb_std',
       'CV_flagcompro_mean', 'CV_flagcompro_sum', 'CV_flagcompro_std',
       'CV_nropaginas_mean', 'CV_nropaginas_max', 'CV_nropaginas_min',
       'CV_nropaginas_sum', 'CV_nropaginas_std', 'CV_nropaginas_quantile',
       'CV_nrodeofertas_mean', 'CV_nrodeofertas_max', 'CV_nrodeofertas_min',
       'CV_nrodeofertas_sum', 'CV_nrodeofertas_std',
       'CV_nrode

In [ ]:
dfcampana = pd.read_csv(s3_manager.s3_getfile(fs_campana)['Body'],dtype ={'idconsultora':str})
dfconsultora = pd.read_csv(s3_manager.s3_getfile(fs_consultora)['Body'],dtype ={'idconsultora':str})

In [346]:
train.flagcontracaratula.unique()

array([ 0,  1,  2,  4, 10,  3, 14,  7,  6,  8, 12, 21,  9, 15, 18, 16])

In [38]:
len(tt)

161421

In [83]:
###############################################################################################
##################################        MODELO        #######################################
###############################################################################################  

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

train = tr_dat.copy()
test = te_dat.copy()

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
#train.fillna(0, inplace=True)

features=[ x for x in train.columns if x not in ['id_persona','codtarget','codmes','margen']]
target='codtarget'